Book Reference: page 221-231 of **Python for Data Analysis Book by Wes McKinney**

# Chapter 8 Data Wrangling: Join, Combine, and Reshape

## Hierarchical Indexing

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.Series(np.random.randn(9),
    index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
    [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.990881
   2    0.605993
   3    1.040223
b  1    0.196632
   3   -0.861067
c  1   -1.782026
   2    2.109911
d  2    0.069414
   3   -0.687726
dtype: float64

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
# partial indexing
data['b']

1    0.196632
3   -0.861067
dtype: float64

In [8]:
data['b':'c']

b  1    0.196632
   3   -0.861067
c  1   -1.782026
   2    2.109911
dtype: float64

In [9]:
data.loc[['b', 'd']]

b  1    0.196632
   3   -0.861067
d  2    0.069414
   3   -0.687726
dtype: float64

In [10]:
data.loc[:, 2]

a    0.605993
c    2.109911
d    0.069414
dtype: float64

---

In [11]:
data.unstack()

,1,2,3
a,-0.990881,0.605993,1.040223
b,0.196632,NaN,-0.861067
c,-1.782026,2.109911,NaN
d,NaN,0.069414,-0.687726


In [12]:
data.unstack().stack()

a  1   -0.990881
   2    0.605993
   3    1.040223
b  1    0.196632
   3   -0.861067
c  1   -1.782026
   2    2.109911
d  2    0.069414
   3   -0.687726
dtype: float64

---

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=[['Ohio', 'Ohio', 'Colorado'],
    ['Green', 'Red', 'Green']])

In [14]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.index.names = ['key1', 'key2']

In [16]:
frame.columns.names = ['state', 'color']

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [20]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting Levels

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [26]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [27]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [28]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [29]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame’s columns

In [30]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
     'c': ['one', 'one', 'one', 'two', 'two',
     'two', 'two'],
     'd': [0, 1, 2, 0, 1, 2, 3]})

In [31]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [32]:
frame2 = frame.set_index(['c', 'd'])

In [33]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [34]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [35]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Dataset
- pandas.merge
- pandas.concat
- combine_first

In [36]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
     'data1': range(7)})

In [37]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
     'data2': range(3)})

In [38]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [39]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a **many-to-one join**; the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. Calling merge with these objects we obtain:

In [40]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


---

In [41]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
     'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [42]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
     'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [43]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')
# default, inner join

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [44]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


---

In [45]:
# many-to-many merges
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
     'data1': range(6)})

In [46]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
     'data2': range(5)})

In [47]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [48]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [49]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [50]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


---

In [52]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
     'key2': ['one', 'two', 'one'],
     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
     'key2': ['one', 'one', 'one', 'two'],
     'rval': [4, 5, 6, 7]})

In [53]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [54]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [55]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


Issue to consider in merge operations
- treatment of overlapping column names
    - solution1 - rename axis labels
    - solution2 - merge has suffixes option to append to overlapping names

In [56]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [57]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7
